## Creating an Initial Population

Now we can create an initial population, by first defining the population size and then selecting from the list of recipes.




In [1]:
import json
import pprint
import random
import math

In [2]:
with open('sample.json', 'r') as file:
    data = json.load(file)
recipes = data['recipes']
pprint.PrettyPrinter(indent=2, depth=3).pprint(recipes[0])

{ 'ingredients': [ { 'amount': 8.0,
                     'ingredient': 'fresh strawberries',
                     'unit': 'oz'},
                   { 'amount': 1.0,
                     'ingredient': 'mochiko flour, sweet rice or glutinous '
                                   'rice flour',
                     'unit': 'cup'},
                   { 'amount': 2.0,
                     'ingredient': 'sugar of choice',
                     'unit': 'tablespoons'},
                   { 'amount': 0.25,
                     'ingredient': 'cornstarch, potato starch or tapioca '
                                   'starch',
                     'unit': 'cup'},
                   { 'amount': 1.0,
                     'ingredient': 'sweetened white bean paste (shiro an)',
                     'unit': 'cup'}],
  'name': 'Strawberry Mochi',
  'rating': 4.71,
  'servings': 8}


In [14]:
all_ingredients = []
for i, recipe in enumerate(recipes):
  all_ingredients.extend(recipe['ingredients'])
  print(i+1, recipe['name'])
print("amount of ingredients", len(all_ingredients))

1 Strawberry Mochi
2 Fresh Mango
3 Applesauce Mochi
4 Sweet Potato
5 Raspberry Chocolate
6 Banana Chocolate
7 Tofu
8 Pumpkin
9 Savory Sweet Corn Mochi
10 Pistachio Butter
11 Black Sesame
12 Purple Sweet Potato Mochi
13 Green Tea / Matcha (with a Twist
14 Almond Milk
15 Crunchy Peanut Butter
16 Blueberry Mochi Ice Cream
17 Mugwort Mochi
18 Pandan Mochi
19 Black Sesame Mochi Muffins
20 Mango Mochi
21 Matcha Mochi Waffles
22 Ube Baked Mochi Donuts
23 Potato Mochi
24 Chocolate Mochi
25 Ube Mochi Muffins
26 Biscoff Baked Mochi Donuts
27 Nutella Mochi
28 Chocolate Mochi Cupcakes
29 Matcha Mochi Muffins
30 Ube Butter Mochi
31 Mochi Waffle Recipe
32 Mochi Bread
33 Chocolate Mochi Muffins
34 Mochi Cookies
35 Chocolate Mochi Donuts
36 Ube Mochi
37 Matcha Baked Mochi Donuts
38 Mochi Pancakes
39 Pandan Donuts
40 Milk Rice Cake
amount of ingredients 305


In [6]:
population_size = 20
population = random.choices(recipes, k=population_size)
pprint.PrettyPrinter(indent=2, depth=4).pprint(population)

[ { 'ingredients': [ { 'amount': 1.0,
                       'ingredient': 'Corn starch',
                       'unit': 'whole'},
                     { 'amount': 90.0,
                       'ingredient': 'glutinous rice flour',
                       'unit': 'grams'},
                     { 'amount': 0.5,
                       'ingredient': 'baking powder',
                       'unit': 'teaspoon'},
                     {'amount': 1.0, 'ingredient': 'of salt', 'unit': 'Pinch'},
                     { 'amount': 30.0,
                       'ingredient': 'unsalted butter',
                       'unit': 'grams'},
                     { 'amount': 30.0,
                       'ingredient': 'brown sugar',
                       'unit': 'grams'},
                     { 'amount': 1.0,
                       'ingredient': 'large egg',
                       'unit': 'whole'},
                     { 'amount': 100.0,
                       'ingredient': 'milk',
                       'unit':

## Evaluating Recipes (Fitness Function)

The following function defines how individuals are evaluated:

Strawberry Mochi
Fresh Mango
Applesauce Mochi
Sweet Potato
Raspberry Chocolate
Banana Chocolate
Tofu
Pumpkin
Savory Sweet Corn Mochi
Pistachio Butter
Black Sesame
Purple Sweet Potato Mochi
Green Tea / Matcha (with a Twist
Almond Milk
Crunchy Peanut Butter
Blueberry Mochi Ice Cream
Mugwort Mochi
Pandan Mochi
Black Sesame Mochi Muffins
Mango Mochi
Matcha Mochi Waffles
Ube Baked Mochi Donuts
Potato Mochi
Chocolate Mochi
Ube Mochi Muffins
Biscoff Baked Mochi Donuts
Nutella Mochi
Chocolate Mochi Cupcakes
Matcha Mochi Muffins
Ube Butter Mochi
Mochi Waffle Recipe
Mochi Bread
Chocolate Mochi Muffins
Mochi Cookies
Chocolate Mochi Donuts
Ube Mochi
Matcha Baked Mochi Donuts
Mochi Pancakes
Pandan Donuts
Milk Rice Cake
